In [101]:
if 'libraries_imported' not in globals():
    libraries_imported = False

if libraries_imported == False:
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import os
    from datetime import datetime, timedelta
    from datetime import datetime
    # from scipy.interpolate import make_interp_splines
    import plotly.graph_objects as go
    from scipy.signal import butter, filtfilt
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    from keras_tuner import RandomSearch, HyperModel
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense
    from tensorflow.keras.callbacks import EarlyStopping
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    from keras_tuner import RandomSearch, HyperModel
    from tensorflow.keras.layers import Input
    import sys
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    import numpy as np
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.preprocessing import RobustScaler
    import tensorflow as tf
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    from sklearn.preprocessing import RobustScaler
    import joblib
    from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
    from sklearn.utils.class_weight import compute_class_weight
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import joblib
    import tensorflow as tf
    from sklearn.preprocessing import RobustScaler
    from sklearn.model_selection import train_test_split
    from sklearn.utils.class_weight import compute_class_weight
    from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
    import numpy as np
    import pandas as pd
    import joblib
    import tensorflow as tf
    from sklearn.model_selection import train_test_split
    from imblearn.over_sampling import SMOTE
    import numpy as np
    import pandas as pd
    import joblib
    import os
    from collections import Counter
    import matplotlib.pyplot as plt
    from sklearn.metrics import (
        accuracy_score, precision_recall_fscore_support,
        confusion_matrix, ConfusionMatrixDisplay
    )
    from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
    from sklearn.preprocessing import RobustScaler
    from imblearn.over_sampling import SMOTE
    from xgboost import XGBClassifier


    # # Display the first few rows
    # import ace_tools as tools  # Importing display tool


Inspect Feature importances


Basic feature engineering -- Add mvoing averages


In [102]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix,
    ConfusionMatrixDisplay
)
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import joblib
import matplotlib.pyplot as plt

# =============================
# Setup portable directories
# =============================
BASE_DIR = Path.cwd()
MODEL_DIR = BASE_DIR / "model_params"
RESULTS_DIR = BASE_DIR / "results"

MODEL_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# =============================
# Load and prepare both datasets
# =============================

# # Dataset 1
# dataset1_df = pd.read_csv("datasets/s_dataset1.csv")
# dataset1_df["date"] = pd.to_datetime(dataset1_df['datetime'], format='%d.%m.%y')
# dataset1_df.drop(columns=["index", "datetime", "f/m_ratio", "fos", "tac"], inplace=True)
# dataset1_df = dataset1_df.drop_duplicates()
# dataset1_df["date"] = pd.to_datetime(dataset1_df["date"], dayfirst=True)

# # Dataset 2
# dataset2_df = pd.read_csv("datasets/c_dataset2.csv")
# dataset2_df['date'] = pd.to_datetime(dataset2_df['date'])
# dataset2_df = dataset2_df.groupby('date').mean()
# dataset2_df = dataset2_df.drop_duplicates()
# dataset2_df.reset_index(inplace=True)
# dataset2_df["methane_percentage"] *= 100


import pandas as pd

# -----------------------------
# Dataset 1
# -----------------------------
dataset1_df = pd.read_csv("datasets/s_dataset1.csv")

# Detect a date-like column
candidates = [c for c in ["datetime", "date", "day_number"] if c in dataset1_df.columns]
if not candidates:
    raise ValueError("No 'date', 'datetime', or 'day_number' column found in dataset 1.")
src_col = candidates[0]

# Build a clean 'date' column
if src_col == "day_number":
    base_date = pd.Timestamp("2020-01-01")  # adjust as needed
    dataset1_df["date"] = base_date + pd.to_timedelta(dataset1_df[src_col], unit="D")
else:
    dataset1_df["date"] = pd.to_datetime(dataset1_df[src_col], dayfirst=True, errors="coerce")

# Drop unwanted columns except the final 'date'
cols_to_drop = ["index", "f/m_ratio", "fos", "tac"]
dataset1_df.drop(columns=[c for c in cols_to_drop if c in dataset1_df.columns], inplace=True)

if src_col != "date" and src_col in dataset1_df.columns:
    dataset1_df.drop(columns=[src_col], inplace=True)

dataset1_df = dataset1_df.drop_duplicates()

# -----------------------------
# Dataset 2
# -----------------------------
dataset2_df = pd.read_csv("datasets/c_dataset2.csv")

# Detect a date-like column
candidates = [c for c in ["datetime", "date", "day_number"] if c in dataset2_df.columns]
if not candidates:
    raise ValueError("No 'date', 'datetime', or 'day_number' column found in dataset 2.")
src_col = candidates[0]

# Convert to datetime or compute from day_number
if src_col == "day_number":
    base_date = pd.Timestamp("2020-01-01")  # adjust as needed
    dataset2_df["date"] = base_date + pd.to_timedelta(dataset2_df[src_col], unit="D")
else:
    dataset2_df["date"] = pd.to_datetime(dataset2_df[src_col], dayfirst=True, errors="coerce")

# Group and clean
dataset2_df = dataset2_df.groupby("date").mean(numeric_only=True)
dataset2_df = dataset2_df.drop_duplicates()
dataset2_df.reset_index(inplace=True)

# Scale methane percentage if present
if "methane_percentage" in dataset2_df.columns:
    dataset2_df["methane_percentage"] *= 100






In [104]:
# ------------------------------
# Imports
# ------------------------------
import os
import random
import joblib
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score

from imblearn.over_sampling import SMOTE
import xgboost as xgb

# ------------------------------
# Config
# ------------------------------
BASE_OUTDIR = "transfer_learning_model_params"   # all outputs go here
os.makedirs(BASE_OUTDIR, exist_ok=True)

RANDOM_STATE = 42
TARGET = "eq_cod"     # numeric target used for binning ONLY (NOT a feature)
N_BINS = 4
BIN_BASE = 500

PARAM_GRID = {
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 400],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
}
N_SPLITS = 3
N_RANDOM_PARAM_ITER = 3

DROP_COLUMNS_LIVE_ONLY = ["h2s_ppm", "t1_cod", "t1_olr", "fostac"]

KEEP_LAST_PERCENT = 20  # default
PRETRAIN_PERCENTS = [5, 10, 20, 40, 60, 80, 100]

# ------------------------------
# Reproducibility
# ------------------------------
os.environ["PYTHONHASHSEED"] = str(RANDOM_STATE)
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

# ------------------------------
# Helpers
# ------------------------------
def round_to_nearest(x, base=500):
    return int(base * round(float(x) / base))

def random_param_sample(grid):
    return {k: random.choice(v) for k, v in grid.items()}

def ensure_columns(df, col_order):
    df = df.copy()
    for c in col_order:
        if c not in df.columns:
            df[c] = np.nan
    extra = [c for c in df.columns if c not in col_order]
    return df[col_order + extra]

def keep_last_percent(df, percent, date_col="date"):
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col]).sort_values(date_col)

    n_keep = int(np.ceil(len(df) * (percent / 100.0)))
    n_keep = max(n_keep, 1)
    return df.iloc[-n_keep:]

def baseline_paths(dataset_tag):
    outdir = os.path.join(BASE_OUTDIR, dataset_tag)
    model_path  = os.path.join(outdir, f"{dataset_tag}_best_model.pkl")
    scaler_path = os.path.join(outdir, f"{dataset_tag}_scaler.pkl")
    feat_path   = os.path.join(outdir, f"{dataset_tag}_feature_cols.pkl")
    bins_path   = os.path.join(outdir, f"{dataset_tag}_bins.pkl")
    labels_path = os.path.join(outdir, f"{dataset_tag}_labels.pkl")
    active_labels_path = os.path.join(outdir, f"{dataset_tag}_active_labels.pkl")
    return outdir, model_path, scaler_path, feat_path, bins_path, labels_path, active_labels_path

def artifacts_exist(dataset_tag):
    _, model_path, scaler_path, feat_path, bins_path, labels_path, active_labels_path = baseline_paths(dataset_tag)
    return all(os.path.exists(p) for p in [
        model_path, scaler_path, feat_path, bins_path, labels_path, active_labels_path
    ])

def make_shared_bins(df_list, target_col, n_bins, base):
    all_vals = pd.concat([df[target_col].dropna() for df in df_list], axis=0)
    vmin, vmax = all_vals.min(), all_vals.max()

    bins = np.linspace(vmin, vmax, n_bins + 1)
    bins = [round_to_nearest(b, base) for b in bins]
    bins[0] = min(bins[0], vmin)
    bins[-1] = max(bins[-1], vmax)

    if len(set(bins)) < len(bins):
        base0 = round_to_nearest(vmin, base)
        bins = [base0 + i * base for i in range(n_bins + 1)]

    labels = [f"{low}-{high}" for low, high in zip(bins[:-1], bins[1:])]
    return bins, labels

def safe_train_test_split(X, y, test_size=0.2, random_state=42):
    """
    Stratify if possible (all classes have >=2 members); otherwise fallback
    to non-stratified split.
    """
    y_series = pd.Series(y)
    counts = y_series.value_counts()
    can_stratify = (counts.min() >= 2) and (len(counts) >= 2)

    if can_stratify:
        return train_test_split(
            X, y, test_size=test_size, stratify=y, random_state=random_state
        )

    # fallback
    return train_test_split(
        X, y, test_size=test_size, stratify=None, random_state=random_state, shuffle=True
    )

# ------------------------------
# Loaders / cleaners
# ------------------------------
def load_flex(path, *, aggregate=False, base_date="2020-01-01",
              methane_col="methane_percentage", scale_methane=True):

    df = pd.read_csv(path)

    src_col = next((c for c in ["datetime", "date", "day_number"] if c in df.columns), None)
    if not src_col:
        raise ValueError(f"No date-like column found in {path}")

    df["date"] = (
        pd.Timestamp(base_date) + pd.to_timedelta(df["day_number"], unit="D")
        if src_col == "day_number"
        else pd.to_datetime(df[src_col], dayfirst=True, errors="coerce")
    )

    if aggregate:
        df = df.groupby("date").mean(numeric_only=True).drop_duplicates().reset_index()
    else:
        drop_cols = [c for c in ["index", "datetime", "day_number", "f/m_ratio", "fos", "tac"]
                     if c in df.columns]
        df.drop(columns=drop_cols, inplace=True, errors="ignore")
        df = df.drop_duplicates()

    if scale_methane and methane_col in df.columns:
        df[methane_col] = df[methane_col] * 100

    return df

def harmonize_columns(sani_df, cook_df):
    sani_cols = ["date"] + [c for c in sani_df.columns if c != "date"]
    sani_df = sani_df.copy()
    cook_df = ensure_columns(cook_df.copy(), sani_cols)

    drop_sani = [c for c in DROP_COLUMNS_LIVE_ONLY if c in sani_df.columns]
    drop_cook = [c for c in DROP_COLUMNS_LIVE_ONLY if c in cook_df.columns]
    if drop_sani:
        sani_df.drop(columns=drop_sani, inplace=True)
    if drop_cook:
        cook_df.drop(columns=drop_cook, inplace=True)
    return sani_df, cook_df

# ------------------------------
# Core pipeline
# ------------------------------
def run_pipeline_for_dataset(df_raw, dataset_tag, *, shared_bins, shared_labels, keep_percent=None):

    (
        outdir, model_path, scaler_path, feat_path, bins_path, labels_path, active_labels_path
    ) = baseline_paths(dataset_tag)
    os.makedirs(outdir, exist_ok=True)

    df = df_raw.copy()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df.sort_values("date", inplace=True)

    kp = KEEP_LAST_PERCENT if keep_percent is None else keep_percent
    df = keep_last_percent(df, kp, date_col="date")

    df = df.interpolate(method="ffill").ffill()

    if TARGET not in df.columns:
        raise ValueError(f"[{dataset_tag}] Target column '{TARGET}' not found.")
    df.dropna(subset=[TARGET], inplace=True)

    # Shared binning
    df["fostac_category"] = pd.cut(
        df[TARGET], bins=shared_bins, labels=shared_labels, include_lowest=True
    )

    df["fostac_category"] = (
        df["fostac_category"]
        .astype("category")
        .cat.set_categories(shared_labels, ordered=True)
    )
    df.dropna(subset=["fostac_category"], inplace=True)

    df["fostac_category"] = df["fostac_category"].cat.remove_unused_categories()
    active_labels = list(df["fostac_category"].cat.categories)

    # X / y
    X = df.select_dtypes(include=[np.number]).drop(columns=[TARGET])
    y = df["fostac_category"].cat.codes

    assert TARGET not in X.columns, "ERROR: eq_cod leaked into training features"
    feature_cols = list(X.columns)

    # If we have too few rows overall, skip (avoid confusing partial artifacts)
    if len(df) < 10 or len(np.unique(y)) < 2:
        print(f"[{dataset_tag}] Skipping (keep_last_percent={kp}) → too few rows/classes after binning.")
        return

    # Train/test split (safe stratify)
    X_train, X_test, y_train, y_test = safe_train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE
    )

    # Determine how many splits we can actually do
    y_train_counts = pd.Series(y_train).value_counts()
    min_class = int(y_train_counts.min()) if len(y_train_counts) else 0
    n_splits_eff = min(N_SPLITS, min_class)

    if n_splits_eff < 2:
        # Not enough per-class samples for CV; train once with default params
        final_scaler = RobustScaler()
        X_train_scaled_full = final_scaler.fit_transform(X_train)

        try:
            sm = SMOTE(random_state=RANDOM_STATE, k_neighbors=2)
            X_train_full_sm, y_train_full_sm = sm.fit_resample(X_train_scaled_full, y_train.astype(int))
        except ValueError:
            X_train_full_sm, y_train_full_sm = X_train_scaled_full, y_train.astype(int)

        final_num_class = int(np.max(y_train_full_sm)) + 1

        final_model = xgb.XGBClassifier(
            objective="multi:softmax",
            num_class=final_num_class,
            eval_metric="mlogloss",
            random_state=RANDOM_STATE,
            **random_param_sample(PARAM_GRID)
        )
        final_model.fit(X_train_full_sm, y_train_full_sm)

        joblib.dump(final_model, model_path)
        joblib.dump(final_scaler, scaler_path)
        joblib.dump(feature_cols, feat_path)
        joblib.dump(shared_bins, bins_path)
        joblib.dump(shared_labels, labels_path)
        joblib.dump(active_labels, active_labels_path)

        print(f"[{dataset_tag}] Saved pretrained baseline (NO CV; keep_last_percent={kp}) → {outdir}")
        return

    # Hyperparam search (CV)
    skf = StratifiedKFold(n_splits=n_splits_eff, shuffle=True, random_state=RANDOM_STATE)
    best_acc = -np.inf
    best_params = None

    for tr_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[tr_idx], X_train.iloc[val_idx]
        y_tr, y_val = pd.Series(y_train).iloc[tr_idx], pd.Series(y_train).iloc[val_idx]

        # Shift fold labels to start at 0
        y_tr_min = int(y_tr.min())
        y_tr_shift = (y_tr - y_tr_min).astype(int)
        y_val_shift = (y_val - y_tr_min).astype(int)

        if not set(y_val_shift.unique()).issubset(set(y_tr_shift.unique())):
            continue

        scaler = RobustScaler()
        X_tr_scaled = scaler.fit_transform(X_tr)
        X_val_scaled = scaler.transform(X_val)

        try:
            sm = SMOTE(random_state=RANDOM_STATE, k_neighbors=2)
            X_tr_sm, y_tr_sm = sm.fit_resample(X_tr_scaled, y_tr_shift)
        except ValueError:
            X_tr_sm, y_tr_sm = X_tr_scaled, y_tr_shift

        fold_num_class = int(np.max(y_tr_sm)) + 1

        for _ in range(N_RANDOM_PARAM_ITER):
            params = random_param_sample(PARAM_GRID)
            model = xgb.XGBClassifier(
                objective="multi:softmax",
                num_class=fold_num_class,
                eval_metric="mlogloss",
                random_state=RANDOM_STATE,
                **params
            )
            model.fit(X_tr_sm, y_tr_sm)
            acc = accuracy_score(y_val_shift, model.predict(X_val_scaled))

            if acc > best_acc:
                best_acc = acc
                best_params = params

    # Final train
    final_scaler = RobustScaler()
    X_train_scaled_full = final_scaler.fit_transform(X_train)

    try:
        sm = SMOTE(random_state=RANDOM_STATE, k_neighbors=2)
        X_train_full_sm, y_train_full_sm = sm.fit_resample(
            X_train_scaled_full, pd.Series(y_train).astype(int)
        )
    except ValueError:
        X_train_full_sm, y_train_full_sm = X_train_scaled_full, pd.Series(y_train).astype(int)

    final_num_class = int(np.max(y_train_full_sm)) + 1

    final_model = xgb.XGBClassifier(
        objective="multi:softmax",
        num_class=final_num_class,
        eval_metric="mlogloss",
        random_state=RANDOM_STATE,
        **(best_params or random_param_sample(PARAM_GRID))
    )
    final_model.fit(X_train_full_sm, y_train_full_sm)

    # Save artifacts
    joblib.dump(final_model, model_path)
    joblib.dump(final_scaler, scaler_path)
    joblib.dump(feature_cols, feat_path)
    joblib.dump(shared_bins, bins_path)
    joblib.dump(shared_labels, labels_path)
    joblib.dump(active_labels, active_labels_path)

    print(f"[{dataset_tag}] Saved pretrained baseline (keep_last_percent={kp}) → {outdir}")

# ------------------------------
# Orchestrate both datasets + train multiple pretrain variants
# ------------------------------
dataset1_df = load_flex("datasets/s_dataset1.csv")
dataset2_df = load_flex("datasets/c_dataset2.csv")

dataset1_df, dataset2_df = harmonize_columns(dataset1_df, dataset2_df)

# Shared bins/labels across BOTH full datasets (fixed reference)
shared_bins, shared_labels = make_shared_bins([dataset1_df, dataset2_df], TARGET, N_BINS, BIN_BASE)

# Train a bank of pretrained baselines at different injection percentages
for pct in PRETRAIN_PERCENTS:
    d1_tag = f"D1_p{pct}"
    d2_tag = f"D2_p{pct}"

    if artifacts_exist(d1_tag):
        print(f"[{d1_tag}] Artifacts already exist, skipping.")
    else:
        run_pipeline_for_dataset(dataset1_df, d1_tag, shared_bins=shared_bins, shared_labels=shared_labels, keep_percent=pct)

    if artifacts_exist(d2_tag):
        print(f"[{d2_tag}] Artifacts already exist, skipping.")
    else:
        run_pipeline_for_dataset(dataset2_df, d2_tag, shared_bins=shared_bins, shared_labels=shared_labels, keep_percent=pct)

print("\nPretrain bank ready. Example paths:")
print(" - transfer_learning_model_params/D1_p20/D1_p20_best_model.pkl")
print(" - transfer_learning_model_params/D1_p20/D1_p20_scaler.pkl")
print(" - transfer_learning_model_params/D2_p20/D2_p20_best_model.pkl")
print(" - transfer_learning_model_params/D2_p20/D2_p20_scaler.pkl")


[D1_p5] Artifacts already exist, skipping.
[D2_p5] Artifacts already exist, skipping.
[D1_p10] Artifacts already exist, skipping.
[D2_p10] Artifacts already exist, skipping.
[D1_p20] Artifacts already exist, skipping.
[D2_p20] Artifacts already exist, skipping.
[D1_p40] Artifacts already exist, skipping.
[D2_p40] Artifacts already exist, skipping.
[D1_p60] Artifacts already exist, skipping.
[D2_p60] Artifacts already exist, skipping.
[D1_p80] Skipping (keep_last_percent=80) → too few rows/classes after binning.
[D2_p80] Artifacts already exist, skipping.
[D1_p100] Skipping (keep_last_percent=100) → too few rows/classes after binning.
[D2_p100] Artifacts already exist, skipping.

Pretrain bank ready. Example paths:
 - transfer_learning_model_params/D1_p20/D1_p20_best_model.pkl
 - transfer_learning_model_params/D1_p20/D1_p20_scaler.pkl
 - transfer_learning_model_params/D2_p20/D2_p20_best_model.pkl
 - transfer_learning_model_params/D2_p20/D2_p20_scaler.pkl


In [105]:
# ===== Baseline (transfer learning + scaler switchable) =====
# Random train/validation split (no temporal ordering)
# Updated: loop over multiple pretrained variants (e.g., D1_p10, D1_p20, ...)

import os, gc, random
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)
from imblearn.over_sampling import SMOTE
import xgboost as xgb

# ------------------ Config ------------------
OUTDIR_BASE = "transfer_learning_model_params/baseline"
os.makedirs(OUTDIR_BASE, exist_ok=True)

BASE_PRETRAIN_DIR = "transfer_learning_model_params"

RANDOM_STATE = 42
RANDOM_STATES = [1, 7, 13, 42, 99]

TARGET = "eq_cod"
N_BINS = 4
BIN_BASE = 500
TRAIN_RATIO = 0.50
CLASS_LABELS = ["LL", "ML", "MH", "HH"]

# NEW: Pretrain variants to try (must exist on disk from your "pretrain bank" script)
PRETRAIN_PERCENTS = [5, 10, 20, 40, 60, 80, 100]

# ------------------ Switches ------------------
ENABLE_TRANSFER_LEARNING = True
ENABLE_PRETRAINED_SCALER = True
ADDITIONAL_ESTIMATORS = 150

if ENABLE_TRANSFER_LEARNING and not ENABLE_PRETRAINED_SCALER:
    raise ValueError(
        "ENABLE_TRANSFER_LEARNING=True requires ENABLE_PRETRAINED_SCALER=True"
    )

# ------------------ Helpers ------------------
def round_to_nearest(x, base=500):
    return int(base * round(float(x) / base))

def random_param_sample(grid):
    return {k: random.choice(v) for k, v in grid.items()}

def build_design_matrix(df, feature_names):
    cols = []
    for name in feature_names:
        if name in df.columns:
            cols.append(df[name])
        else:
            cols.append(pd.Series(0.0, index=df.index, name=name))
    return pd.concat(cols, axis=1)

def bin_and_encode(df, target_col, n_bins=4, base=500, class_labels=None):
    if class_labels is None:
        class_labels = ["LL", "ML", "MH", "HH"]

    vmin, vmax = df[target_col].min(), df[target_col].max()
    bins = np.linspace(vmin, vmax, n_bins + 1)
    bins = [round_to_nearest(b, base) for b in bins]
    bins[0] = min(bins[0], vmin)
    bins[-1] = max(bins[-1], vmax)

    df = df.copy()
    df["target_label"] = pd.cut(
        df[target_col], bins=bins, labels=class_labels, include_lowest=True
    )
    df = df.dropna(subset=["target_label"])
    df["target_label"] = (
        df["target_label"]
        .astype("category")
        .cat.set_categories(class_labels, ordered=True)
    )
    return df, df["target_label"].cat.codes, bins

def prepare_df_for_baseline(raw_df, dayfirst=True):
    df = raw_df.copy()
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce", dayfirst=dayfirst)
    df = df.interpolate(method="ffill").ffill()
    return df.dropna(subset=[TARGET])

def smote_safe(X, y, random_state=42):
    y = pd.Series(y)
    counts = Counter(y)
    if not counts:
        return X, y.values

    maj = max(counts.values())
    minority_counts = [c for c in counts.values() if c > 1]
    min_count = min(minority_counts) if minority_counts else 0

    k_neighbors = max(1, min(3, min_count - 1)) if min_count > 0 else 1
    sampling_strategy = {cls: maj for cls, cnt in counts.items() if cnt > 1}

    if sampling_strategy and min_count > 0:
        sm = SMOTE(
            random_state=random_state,
            k_neighbors=k_neighbors,
            sampling_strategy=sampling_strategy
        )
        return sm.fit_resample(X, y)

    return X, y.values

def pretrained_artifacts_exist(pretrained_tag):
    pre_dir = os.path.join(BASE_PRETRAIN_DIR, pretrained_tag)
    return (
        os.path.exists(os.path.join(pre_dir, f"{pretrained_tag}_best_model.pkl")) and
        os.path.exists(os.path.join(pre_dir, f"{pretrained_tag}_scaler.pkl")) and
        os.path.exists(os.path.join(pre_dir, f"{pretrained_tag}_feature_cols.pkl"))
    )

def load_pretrained_artifacts(pretrained_tag):
    pre_dir = os.path.join(BASE_PRETRAIN_DIR, pretrained_tag)

    model = joblib.load(os.path.join(pre_dir, f"{pretrained_tag}_best_model.pkl"))
    scaler = joblib.load(os.path.join(pre_dir, f"{pretrained_tag}_scaler.pkl"))
    feature_cols = joblib.load(os.path.join(pre_dir, f"{pretrained_tag}_feature_cols.pkl"))

    return model.get_booster(), scaler, feature_cols

# ------------------ Baseline ------------------
def baseline_direction(tgt_tag, tgt_df, pretrained_tag=None):
    print(f"\n====================")
    print(f"Baseline (random split): {tgt_tag}")
    print(f"====================")

    random.seed(RANDOM_STATE)
    np.random.seed(RANDOM_STATE)

    use_transfer = ENABLE_TRANSFER_LEARNING and (pretrained_tag is not None)
    use_scaler   = ENABLE_PRETRAINED_SCALER and (pretrained_tag is not None)

    booster = None
    pretrained_scaler = None
    pretrained_feature_cols = None

    if use_transfer or use_scaler:
        booster, pretrained_scaler, pretrained_feature_cols = load_pretrained_artifacts(
            pretrained_tag
        )

    outdir = os.path.join(OUTDIR_BASE, tgt_tag)
    os.makedirs(outdir, exist_ok=True)

    model_path  = os.path.join(outdir, f"{tgt_tag}_baseline_model.pkl")
    scaler_path = os.path.join(outdir, f"{tgt_tag}_baseline_scaler.pkl")
    sensor_csv  = os.path.join(outdir, f"{tgt_tag}_sensor_predictions_baseline.csv")

    # 1) Prepare + encode
    df = prepare_df_for_baseline(tgt_df)
    df, y_all, _ = bin_and_encode(df, TARGET, N_BINS, BIN_BASE, CLASS_LABELS)

    if use_transfer or use_scaler:
        feature_cols = pretrained_feature_cols
    else:
        feature_cols = [c for c in df.columns if c not in ["date", TARGET, "target_label"]]

    X_all = build_design_matrix(df, feature_cols)

    # ---- RANDOM SPLIT ----
    rng = np.random.default_rng(RANDOM_STATE)
    idx = rng.permutation(len(df))
    train_size = int(TRAIN_RATIO * len(df))

    train_idx = idx[:train_size]
    val_idx   = idx[train_size:]

    X_train_raw = X_all.iloc[train_idx].copy()
    y_train     = y_all.iloc[train_idx].copy()

    X_val_raw = X_all.iloc[val_idx].copy()
    y_val     = y_all.iloc[val_idx].copy()

    val_dates = df.iloc[val_idx]["date"]

    # 2) Hyperparam search
    if use_transfer:
        param_grid = {
            "max_depth": [3, 4, 5, 6],
            "learning_rate": [0.01, 0.05, 0.1],
            "subsample": [0.8, 1.0],
            "colsample_bytree": [0.8, 1.0],
            "gamma": [0.0, 0.1, 0.2],
        }
    else:
        param_grid = {
            "max_depth": [3, 4, 5, 6],
            "learning_rate": [0.01, 0.05, 0.1],
            "n_estimators": [50, 100, 200],
            "subsample": [0.8, 1.0],
            "colsample_bytree": [0.8, 1.0],
            "gamma": [0.0, 0.1, 0.2],
        }

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    best_params, best_cv_acc = None, -np.inf

    for _ in range(10):
        params = random_param_sample(param_grid)
        scores = []

        for tr, va in skf.split(X_train_raw, y_train):
            if use_scaler:
                scaler = pretrained_scaler
                X_tr = scaler.transform(X_train_raw.iloc[tr])
                X_va = scaler.transform(X_train_raw.iloc[va])
            else:
                scaler = RobustScaler()
                X_tr = scaler.fit_transform(X_train_raw.iloc[tr])
                X_va = scaler.transform(X_train_raw.iloc[va])

            X_tr_sm, y_tr_sm = smote_safe(X_tr, y_train.iloc[tr], RANDOM_STATE)

            if use_transfer:
                model = xgb.XGBClassifier(
                    objective="multi:softmax",
                    num_class=len(CLASS_LABELS),
                    eval_metric="mlogloss",
                    random_state=RANDOM_STATE,
                    nthread=1,
                    tree_method="hist",
                    n_estimators=ADDITIONAL_ESTIMATORS,
                    **params
                )
                model.fit(X_tr_sm, y_tr_sm, xgb_model=booster)
            else:
                model = xgb.XGBClassifier(
                    objective="multi:softmax",
                    num_class=len(CLASS_LABELS),
                    eval_metric="mlogloss",
                    random_state=RANDOM_STATE,
                    nthread=1,
                    tree_method="hist",
                    **params
                )
                model.fit(X_tr_sm, y_tr_sm)

            scores.append(accuracy_score(y_train.iloc[va], model.predict(X_va)))

        mean_score = float(np.mean(scores))
        if mean_score > best_cv_acc:
            best_cv_acc = mean_score
            best_params = params

    print(f"Best CV accuracy: {best_cv_acc:.4f}")
    print(f"Best hyperparameters: {best_params}")

    # 3) Final train
    if use_scaler:
        scaler = pretrained_scaler
        X_train_s = scaler.transform(X_train_raw)
        X_val_s   = scaler.transform(X_val_raw)
    else:
        scaler = RobustScaler().fit(X_train_raw)
        X_train_s = scaler.transform(X_train_raw)
        X_val_s   = scaler.transform(X_val_raw)

    X_train_sm, y_train_sm = smote_safe(X_train_s, y_train, RANDOM_STATE)

    if use_transfer:
        final_model = xgb.XGBClassifier(
            objective="multi:softmax",
            num_class=len(CLASS_LABELS),
            eval_metric="mlogloss",
            random_state=RANDOM_STATE,
            nthread=1,
            tree_method="hist",
            n_estimators=ADDITIONAL_ESTIMATORS,
            **(best_params or {})
        )
        final_model.fit(X_train_sm, y_train_sm, xgb_model=booster)
    else:
        final_model = xgb.XGBClassifier(
            objective="multi:softmax",
            num_class=len(CLASS_LABELS),
            eval_metric="mlogloss",
            random_state=RANDOM_STATE,
            nthread=1,
            tree_method="hist",
            **(best_params or {})
        )
        final_model.fit(X_train_sm, y_train_sm)

    # 4) Evaluation (REDUCED METRICS)
    y_pred = final_model.predict(X_val_s)

    bacc = balanced_accuracy_score(y_val, y_pred)
    mae_bins = float(np.mean(np.abs(y_val.values - y_pred)))
    rmse_bins = float(np.sqrt(np.mean((y_val.values - y_pred) ** 2)))

    print(f"Hold-out balanced accuracy:{bacc:.4f}")
    print(f"Hold-out MAE (bin steps):  {mae_bins:.4f}")
    print(f"Hold-out RMSE (bin steps): {rmse_bins:.4f}")

    # 5) Save
    sensor_df = pd.DataFrame({
        "date": val_dates,
        "pred_bin_code": y_pred,
        "pred_bin_label": pd.Categorical(
            y_pred, categories=[0, 1, 2, 3]
        ).rename_categories(CLASS_LABELS).astype(str)
    }).set_index("date")

    sensor_df.to_csv(sensor_csv)
    joblib.dump(final_model, model_path)
    joblib.dump(scaler, scaler_path)

    print(f"Saved model → {model_path}")
    print(f"Saved scaler → {scaler_path}")
    print(f"Saved predictions → {sensor_csv}")

    del final_model
    gc.collect()

    return {
        "run_tag": tgt_tag,
        "random_state": RANDOM_STATE,
        "transfer": bool(use_transfer),
        "use_scaler": bool(use_scaler),
        "pretrained_tag": pretrained_tag if pretrained_tag is not None else "",
        "train_ratio": TRAIN_RATIO,
        "best_cv_acc": float(best_cv_acc),
        "holdout_bacc": float(bacc),
        "holdout_mae_bins": float(mae_bins),
        "holdout_rmse_bins": float(rmse_bins),
    }

# ------------------ Run (vary RANDOM_STATE; try multiple pretrained variants) ------------------

results = []

# Always run scratch once per seed (reference), then sweep pretrained variants
for rs in RANDOM_STATES:
    RANDOM_STATE = rs

    print(f"\n\n#############################################")
    print(f"########## RANDOM_STATE = {RANDOM_STATE} ##########")
    print(f"#############################################\n")

    # 1) NON-transfer (scratch)
    ENABLE_TRANSFER_LEARNING = False
    ENABLE_PRETRAINED_SCALER = False
    results.append(baseline_direction(f"D1_scratch_rs{rs}", dataset1_df))
    results.append(baseline_direction(f"D2_scratch_rs{rs}", dataset2_df))

    # 2) Transfer learning: sweep over pretrain bank
    ENABLE_TRANSFER_LEARNING = True
    ENABLE_PRETRAINED_SCALER = True

    for pct in PRETRAIN_PERCENTS:
        d1_pre = f"D1_p{pct}"
        d2_pre = f"D2_p{pct}"

        # D1 target warm-started from D2 pretrain variant
        if pretrained_artifacts_exist(d2_pre):
            results.append(
                baseline_direction(
                    f"D1_from_{d2_pre}_rs{rs}",
                    dataset1_df,
                    pretrained_tag=d2_pre
                )
            )
        else:
            print(f"[SKIP] Missing pretrained artifacts for {d2_pre}")

        # D2 target warm-started from D1 pretrain variant
        if pretrained_artifacts_exist(d1_pre):
            results.append(
                baseline_direction(
                    f"D2_from_{d1_pre}_rs{rs}",
                    dataset2_df,
                    pretrained_tag=d1_pre
                )
            )
        else:
            print(f"[SKIP] Missing pretrained artifacts for {d1_pre}")

# ------------------ Summary stats (REDUCED) ------------------

results_df = pd.DataFrame(results)

# dataset: D1/D2 target (from run_tag prefix)
results_df["dataset"] = results_df["run_tag"].str.extract(r"^(D1|D2)")

# mode: scratch/transfer
results_df["mode"] = np.where(results_df["transfer"], "transfer", "scratch")

# pretrain_pct: parse pXX from pretrained_tag if present
results_df["pretrain_pct"] = (
    results_df["pretrained_tag"]
    .str.extract(r"_p(\d+)")
    .astype(float)
)

print("\n\n====================")
print("All runs (raw)")
print("====================")
print(results_df)

metrics = ["holdout_bacc", "holdout_mae_bins", "holdout_rmse_bins"]

# Summaries:
# 1) scratch baseline (per dataset)
scratch_summary = (
    results_df[results_df["mode"] == "scratch"]
    .groupby(["dataset"])[metrics]
    .agg(["mean", "std", "min", "max"])
)

print("\n\n====================")
print("Scratch summary by dataset (mean/std/min/max)")
print("====================")
print(scratch_summary)

# 2) transfer sweep (dataset + pretrain_pct)
transfer_summary = (
    results_df[results_df["mode"] == "transfer"]
    .groupby(["dataset", "pretrain_pct"])[metrics]
    .agg(["mean", "std", "min", "max"])
    .sort_index()
)

print("\n\n====================")
print("Transfer summary by dataset + pretrained % (mean/std/min/max)")
print("====================")
print(transfer_summary)

# Optional: save full results
summary_csv = os.path.join(OUTDIR_BASE, "run_summary_across_seeds_and_pretrain_pct.csv")
results_df.to_csv(summary_csv, index=False)
print(f"\nSaved aggregated run metrics → {summary_csv}")




#############################################
########## RANDOM_STATE = 1 ##########
#############################################


Baseline (random split): D1_scratch_rs1
Best CV accuracy: 0.5455
Best hyperparameters: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 200, 'subsample': 1.0, 'colsample_bytree': 0.8, 'gamma': 0.2}
Hold-out balanced accuracy:0.6643
Hold-out MAE (bin steps):  0.4545
Hold-out RMSE (bin steps): 0.8704
Saved model → transfer_learning_model_params/baseline/D1_scratch_rs1/D1_scratch_rs1_baseline_model.pkl
Saved scaler → transfer_learning_model_params/baseline/D1_scratch_rs1/D1_scratch_rs1_baseline_scaler.pkl
Saved predictions → transfer_learning_model_params/baseline/D1_scratch_rs1/D1_scratch_rs1_sensor_predictions_baseline.csv

Baseline (random split): D2_scratch_rs1
Best CV accuracy: 0.6190
Best hyperparameters: {'max_depth': 6, 'learning_rate': 0.01, 'n_estimators': 100, 'subsample': 1.0, 'colsample_bytree': 0.8, 'gamma': 0.2}
Hold-out balanced accu

In [106]:
from IPython.display import display

metrics = ["holdout_bacc", "holdout_mae_bins", "holdout_rmse_bins"]

# transfer_summary is the one that includes ALL pretrained variants (dataset x pretrain_pct)
# It was created like:
# transfer_summary = results_df[results_df["mode"]=="transfer"].groupby(["dataset","pretrain_pct"])[metrics].agg(["mean","std","min","max"])

# 1) Flatten MultiIndex columns -> "holdout_bacc_mean", ...
transfer_summary_flat = transfer_summary.copy()
transfer_summary_flat.columns = [f"{m}_{stat}" for (m, stat) in transfer_summary_flat.columns]
transfer_summary_flat = transfer_summary_flat.reset_index()

# 2) Build a compact table
refined = transfer_summary_flat.loc[:, [
    "dataset", "pretrain_pct",
    "holdout_bacc_mean", "holdout_bacc_std",
    "holdout_mae_bins_mean", "holdout_mae_bins_std",
    "holdout_rmse_bins_mean"
]].copy()

refined = refined.round(4)

refined["BAcc (mean±std)"] = refined["holdout_bacc_mean"].map(lambda x: f"{x:.3f}") + " ± " + refined["holdout_bacc_std"].map(lambda x: f"{x:.3f}")
refined["MAE bins (mean±std)"] = refined["holdout_mae_bins_mean"].map(lambda x: f"{x:.3f}") + " ± " + refined["holdout_mae_bins_std"].map(lambda x: f"{x:.3f}")
refined["RMSE bins (mean)"] = refined["holdout_rmse_bins_mean"].map(lambda x: f"{x:.3f}")

# nicer label for percent
refined["pretrain_pct"] = refined["pretrain_pct"].astype("Int64")

refined = refined[["dataset", "pretrain_pct", "BAcc (mean±std)", "MAE bins (mean±std)", "RMSE bins (mean)"]]
refined = refined.sort_values(["dataset", "pretrain_pct"])

display(refined)


,dataset,pretrain_pct,BAcc (mean±std),MAE bins (mean±std),RMSE bins (mean)
0,D1,5,0.648 ± 0.093,0.470 ± 0.068,0.857
1,D1,10,0.645 ± 0.088,0.488 ± 0.087,0.868
2,D1,20,0.639 ± 0.094,0.470 ± 0.094,0.865
3,D1,40,0.589 ± 0.066,0.542 ± 0.053,0.934
4,D1,60,0.627 ± 0.084,0.482 ± 0.096,0.866
5,D1,80,0.619 ± 0.093,0.497 ± 0.073,0.895
6,D1,100,0.645 ± 0.073,0.473 ± 0.062,0.865
7,D2,5,0.556 ± 0.100,0.625 ± 0.145,1.033
8,D2,10,0.572 ± 0.063,0.609 ± 0.121,1.033
9,D2,20,0.544 ± 0.079,0.644 ± 0.113,1.069


In [107]:
# ============================================================
# Compare BEST transfer (by pretrain_pct) vs SCRATCH baseline
# Assumes you already have a results_df with columns:
#   - dataset (e.g. "D1","D2")
#   - mode ("scratch" or "transfer")
#   - pretrain_pct (number for transfer rows; NaN for scratch rows is OK)
#   - holdout_bacc, holdout_mae_bins, holdout_rmse_bins
# ============================================================

import os
import numpy as np
import pandas as pd
from IPython.display import display

OUTDIR_BASE = "transfer_learning_model_params/baseline"
os.makedirs(OUTDIR_BASE, exist_ok=True)

METRICS = ["holdout_bacc", "holdout_mae_bins", "holdout_rmse_bins"]

def _mean_std(df, col):
    return float(df[col].mean()), float(df[col].std(ddof=1)) if len(df) > 1 else (float(df[col].mean()), float("nan"))

# ---------------------------
# 1) Scratch summary (per dataset)
# ---------------------------
scratch_df = results_df[results_df["mode"] == "scratch"].copy()

scratch_summary = (
    scratch_df
    .groupby("dataset")[METRICS]
    .agg(["mean", "std", "min", "max"])
)

# Flatten columns -> holdout_bacc_mean, ...
scratch_summary.columns = [f"{m}_{stat}" for (m, stat) in scratch_summary.columns]
scratch_summary = scratch_summary.reset_index()

# ---------------------------
# 2) Transfer summary (per dataset, pretrain_pct)
# ---------------------------
transfer_df = results_df[results_df["mode"] == "transfer"].copy()

# Ensure pretrain_pct is numeric
transfer_df["pretrain_pct"] = pd.to_numeric(transfer_df["pretrain_pct"], errors="coerce")

transfer_summary = (
    transfer_df
    .dropna(subset=["pretrain_pct"])
    .groupby(["dataset", "pretrain_pct"])[METRICS]
    .agg(["mean", "std", "min", "max"])
)

transfer_summary.columns = [f"{m}_{stat}" for (m, stat) in transfer_summary.columns]
transfer_summary = transfer_summary.reset_index()

# ---------------------------
# 3) Pick "best transfer" per dataset (highest mean BAcc)
# ---------------------------
best_transfer_idx = (
    transfer_summary
    .groupby("dataset")["holdout_bacc_mean"]
    .idxmax()
)

best_transfer = transfer_summary.loc[best_transfer_idx].copy()
best_transfer["mode"] = "transfer_best"

# ---------------------------
# 4) Build comparison table: scratch vs best transfer
# ---------------------------
scratch_comp = scratch_summary.copy()
scratch_comp["pretrain_pct"] = np.nan
scratch_comp["mode"] = "scratch"

# Align columns
cols_keep = [
    "dataset", "mode", "pretrain_pct",
    "holdout_bacc_mean", "holdout_bacc_std",
    "holdout_mae_bins_mean", "holdout_mae_bins_std",
    "holdout_rmse_bins_mean", "holdout_rmse_bins_std",
]

scratch_comp = scratch_comp.loc[:, cols_keep]
best_transfer = best_transfer.loc[:, cols_keep]

comparison = pd.concat([scratch_comp, best_transfer], ignore_index=True)

# Nicely formatted strings
comparison_fmt = comparison.copy()
comparison_fmt["pretrain_pct"] = comparison_fmt["pretrain_pct"].astype("Int64")

comparison_fmt["BAcc (mean±std)"] = (
    comparison_fmt["holdout_bacc_mean"].map(lambda x: f"{x:.3f}") +
    " ± " +
    comparison_fmt["holdout_bacc_std"].map(lambda x: "nan" if pd.isna(x) else f"{x:.3f}")
)

comparison_fmt["MAE bins (mean±std)"] = (
    comparison_fmt["holdout_mae_bins_mean"].map(lambda x: f"{x:.3f}") +
    " ± " +
    comparison_fmt["holdout_mae_bins_std"].map(lambda x: "nan" if pd.isna(x) else f"{x:.3f}")
)

comparison_fmt["RMSE bins (mean±std)"] = (
    comparison_fmt["holdout_rmse_bins_mean"].map(lambda x: f"{x:.3f}") +
    " ± " +
    comparison_fmt["holdout_rmse_bins_std"].map(lambda x: "nan" if pd.isna(x) else f"{x:.3f}")
)

comparison_fmt = comparison_fmt.loc[:, [
    "dataset", "mode", "pretrain_pct",
    "BAcc (mean±std)", "MAE bins (mean±std)", "RMSE bins (mean±std)"
]].sort_values(["dataset", "mode", "pretrain_pct"], na_position="first")

# ---------------------------
# 5) Also compute deltas (best transfer - scratch) per dataset
# ---------------------------
scratch_means = scratch_comp.set_index("dataset")[["holdout_bacc_mean", "holdout_mae_bins_mean", "holdout_rmse_bins_mean"]]
best_means = best_transfer.set_index("dataset")[["holdout_bacc_mean", "holdout_mae_bins_mean", "holdout_rmse_bins_mean"]]

delta = (best_means - scratch_means).reset_index()
delta = delta.rename(columns={
    "holdout_bacc_mean": "delta_bacc",
    "holdout_mae_bins_mean": "delta_mae_bins",
    "holdout_rmse_bins_mean": "delta_rmse_bins",
})
delta["delta_bacc"] = delta["delta_bacc"].map(lambda x: f"{x:+.3f}")
delta["delta_mae_bins"] = delta["delta_mae_bins"].map(lambda x: f"{x:+.3f}")
delta["delta_rmse_bins"] = delta["delta_rmse_bins"].map(lambda x: f"{x:+.3f}")

# Add chosen best pct for context
best_pct = best_transfer.set_index("dataset")["pretrain_pct"].astype("Int64").rename("best_pretrain_pct").reset_index()
delta = delta.merge(best_pct, on="dataset", how="left")
delta = delta[["dataset", "best_pretrain_pct", "delta_bacc", "delta_mae_bins", "delta_rmse_bins"]].sort_values("dataset")

# ---------------------------
# 6) Display + save
# ---------------------------
print("\n=== Scratch vs Best Transfer (per dataset) ===")
display(comparison_fmt)

print("\n=== Deltas (Best Transfer - Scratch) ===")
display(delta)

out_csv = os.path.join(OUTDIR_BASE, "scratch_vs_best_transfer.csv")
comparison_fmt.to_csv(out_csv, index=False)
print(f"\nSaved → {out_csv}")

out_csv2 = os.path.join(OUTDIR_BASE, "scratch_vs_best_transfer_deltas.csv")
delta.to_csv(out_csv2, index=False)
print(f"Saved → {out_csv2}")



=== Scratch vs Best Transfer (per dataset) ===


,dataset,mode,pretrain_pct,BAcc (mean±std),MAE bins (mean±std),RMSE bins (mean±std)
0,D1,scratch,<NA>,0.645 ± 0.096,0.448 ± 0.068,0.821 ± 0.082
2,D1,transfer_best,5,0.648 ± 0.093,0.470 ± 0.068,0.857 ± 0.100
1,D2,scratch,<NA>,0.608 ± 0.097,0.459 ± 0.131,0.834 ± 0.159
3,D2,transfer_best,10,0.572 ± 0.063,0.609 ± 0.122,1.033 ± 0.149



=== Deltas (Best Transfer - Scratch) ===


,dataset,best_pretrain_pct,delta_bacc,delta_mae_bins,delta_rmse_bins
0,D1,5,+0.003,+0.021,+0.036
1,D2,10,-0.036,+0.150,+0.199



Saved → transfer_learning_model_params/baseline/scratch_vs_best_transfer.csv
Saved → transfer_learning_model_params/baseline/scratch_vs_best_transfer_deltas.csv
